In [2]:
# NOTE ### replace value with the proper filepath to the lifetime_prb2020-master folder
# local_path_to_lifetimeprb2020 = "/Users/newberry/Code/research/yates_code/lifetime_prb2020-master/"
# local_path_to_lifetimeprb2020 = "/mnt/c/Users/Alex/Code/research/yates_code/lifetime_prb2020-master/"
local_path_to_lifetimeprb2020 = "/home/data/an3205/Code/research/yates_code/lifetime_prb2020-master/"

"/home/data/an3205/Code/research/yates_code/lifetime_prb2020-master/"

In [3]:
using LinearAlgebra
using Dates
using PyPlot
using FileIO

using KrylovKit
using SparseArrays
using JLD
using Roots

using LaTeXStrings
using Plots

push!(LOAD_PATH, local_path_to_lifetimeprb2020*"/ed_main/src")

Base.load_path()

using ED
#Hamiltonian = ED.H_Delta(2, 1.0, 1.0, 1.0, 1.0)

# Generating Ainf data

In [13]:
L_list = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
gamma_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
jz_list = [0, 0.01, 0.03, 0.05, 0.07, 0.09, 0.1, 0.2, 0.3]
jx = 1.0
g = 0.3

# Logarithmic time scale for Ainf, default from 0.1 to 10000
t_list = 10.0 .^(range(-1,stop=4,length=200))

Ainfs_dict = Dict()

for (L_i, L) in enumerate(L_list)
    println(L)
    for (jz_i, jz) in enumerate(jz_list)
        for (gamma_i, gamma) in enumerate(gamma_list)
            Ainf = ED.get_Ainf(L, g, jx, jz, gamma, t_list)
            Ainfs_dict[L, jz, gamma] = Ainf
        end
    end
end

2
3d: ti = 200
4d: ti = 200


Excessive output truncated after 524301 bytes.

In [17]:
Ainfs_dict

Dict{Any, Any} with 891 entries:
  (3, 0.3, 0.1)   => [0.992381, 0.991451, 0.990407, 0.989237, 0.987926, 0.98645…
  (7, 0.05, 0.3)  => [0.995713, 0.995189, 0.994601, 0.993941, 0.993201, 0.99237…
  (7, 0.07, 0.8)  => [0.99791, 0.997654, 0.997368, 0.997047, 0.996687, 0.996283…
  (7, 0.01, 0.8)  => [0.998005, 0.997761, 0.997488, 0.997182, 0.996838, 0.99645…
  (7, 0.3, 0.5)   => [0.99517, 0.99458, 0.993919, 0.993177, 0.992346, 0.991414,…
  (8, 0.09, 0.2)  => [0.994853, 0.994224, 0.993518, 0.992726, 0.991838, 0.99084…
  (8, 0.1, 0.9)   => [0.997958, 0.997708, 0.997428, 0.997115, 0.996763, 0.99636…
  (10, 0.2, 0.1)  => [0.993372, 0.992563, 0.991654, 0.990635, 0.989493, 0.98821…
  (5, 0.05, 0.5)  => [0.99691, 0.996533, 0.996109, 0.995634, 0.995101, 0.994504…
  (2, 0.3, 0.2)   => [0.99322, 0.992391, 0.991461, 0.990417, 0.989247, 0.987935…
  (7, 0.0, 0.7)   => [0.997758, 0.997484, 0.997177, 0.996832, 0.996446, 0.99601…
  (3, 0.2, 0.6)   => [0.996613, 0.996199, 0.995735, 0.995215, 0.994631, 0.99

In [ ]:
using JLD

save("XXX.jld", 
    "Ainfs_dict", Ainfs_dict, 
    "t_list", t_list, 
    "L_list", L_list, 
    "jz_list", jz_list,
    "gamma_list", gamma_list, 
    "jx", 1.0,
    "g", g)

In [ ]:
test = load( "Ainfs_dict_v2.jld"  )
test["Ainfs_dict"] == Ainfs_dict

# Generating Ainf data 2

In [5]:
ED.get_Ainf

get_Ainf (generic function with 1 method)

In [ ]:
function get_Ayy_inf(L::Int64,g::Float64,jx::Float64,jz::Float64, gamma::Float64,times::AbstractArray)
    #=
    Unoptimized autocorrelation for y1-y1
    =#
    ainfs = zeros(Complex{Float64}, length(times))
    sigx = sigX(L,1)
    sigz = sigZ(L,1)
    sigy = im.*(sigz*sigx)
    us,vs = eigen(H_Delta(L,jx,jz,g,gamma))
    sigyV = vs'*sigy*vs
    for ti in 1:length(times)
        print("ed: ti = $ti\r")
        U = Diagonal(exp.(-im*times[ti].*us))
        ainfs[ti] = 1/(2^L)*tr(U'*sigyV*U*sigyV)
    end
    return real.(ainfs)
end

function get_Ainf(L::Int64,g::Float64,jx::Float64,jz::Float64, gamma::Float64,times::AbstractArray)
    #=
    This is partially optimized.
    =#
    ainfs = zeros(Complex{Float64}, length(times))
    sigx = Complex{Float64}.(sigX(L,1))
    us,vs = get_uvs(L,g,jx,jz,gamma)
    sigxV = vs'*sigx*vs
    for ti in 1:length(times)
        print("ed: ti = $ti\r")
        U = Diagonal(exp.(-im*times[ti].*us))
        ainfs[ti] = 1/(2^L)*tr(U'*sigxV*U*sigxV)
    end
    return real.(ainfs)
end

In [14]:
L_list = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
gamma_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
jz_list = [0, 0.01, 0.03, 0.05, 0.07, 0.09, 0.1, 0.2, 0.3]
jx = 1.0
g = 0.3

# Logarithmic time scale for Ainf, default from 0.1 to 10000
t_list = 10.0 .^(range(-1,stop=4,length=200))

Ainfs_dict = Dict()

for (L_i, L) in enumerate(L_list)
    println(L)
    for (jz_i, jz) in enumerate(jz_list)
        for (gamma_i, gamma) in enumerate(gamma_list)
            Ainf = ED.get_Ayy_inf(L, g, jx, jz, gamma, t_list)
            Ainfs_dict[L, jz, gamma] = Ainf
        end
    end
end

2


3


4


Excessive output truncated after 524301 bytes.

In [12]:
using JLD

In [13]:
save("data/Ainfs_dict_sigy_v2.jld", 
    "Ainfs_dict", Ainfs_dict, 
    "t_list", t_list, 
    "L_list", L_list, 
    "jz_list", jz_list,
    "gamma_list", gamma_list, 
    "jx", 1.0,
    "g", g)

In [9]:
test = load( "Ainfs_dict_sigy_v1.jld"  )

Dict{String, Any} with 7 entries:
  "g"          => 0.3
  "jz_list"    => [0.0, 0.01, 0.03, 0.05, 0.07, 0.09, 0.1, 0.2, 0.3]
  "L_list"     => [2, 3, 4, 5, 6, 7, 8, 9]
  "jx"         => 1.0
  "Ainfs_dict" => Dict{Any, Any}((4, 0.3, 0.2)=>[0.989245, 0.987932, 0.98646, 0…
  "gamma_list" => [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
  "t_list"     => [0.1, 0.105956, 0.112267, 0.118953, 0.126038, 0.133545, 0.141…

In [10]:
test["Ainfs_dict"] == Ainfs_dict

true